# 특정한 복셀의 거리, 속도, 가속도 그래프 추출

다양한 역학적 정보를 그래프로 추출하고 이미지와 csv로 저장합니다.

### Import Libraries

In [178]:
import torch
import numpy as np
import pickle
import matplotlib.pyplot as plt
import glob
import gc
import os
import time
import pandas as pd

from vit_rollout import VITAttentionRollout

### Definition

In [179]:
def plot_dis(kinematic, save_name):
    fig, ax = plt.subplots()

    ax.set_title(save_name)

    ax.set_ylim(0, 21)

    ax.set_xlabel('frame number')
    ax.set_ylabel('distance')

    ax.set_xticks(kinematic['number'])
    ax.set_yticks([y for y in range(0, 21)])

    ax.plot(np.array(kinematic['number']), np.array(kinematic['dis']))

    ax.grid()

    fig.set_figwidth(2000/fig.dpi)
    fig.set_figheight(700/fig.dpi)
    fig.savefig(save_name+'.png')

def plot_dis_avg(left_kinematic, right_kinematic, save_name):
    fig, ax = plt.subplots()

    ax.set_title(save_name)

    ax.set_ylim(0, 21)

    ax.set_xlabel('frame number')
    ax.set_ylabel('distance')

    ax.set_xticks(left_kinematic['number'])
    ax.set_yticks([y for y in range(0, 21)])

    ax.plot(np.array(left_kinematic['number']), (np.array(left_kinematic['dis'])+np.array(right_kinematic['dis']))/2)

    ax.grid()

    fig.set_figwidth(2000/fig.dpi)
    fig.set_figheight(700/fig.dpi)
    fig.savefig(save_name+'.png')

In [180]:
def plot_velo(kinematic, save_name):
    fig, ax = plt.subplots()

    ax.set_title(save_name)

    ax.set_ylim(-210, 210)

    ax.set_xlabel('frame number')
    ax.set_ylabel('velocity')

    ax.set_xticks(kinematic['number'])
    ax.set_yticks([y*10 for y in range(-20, 21)])

    ax.plot(np.array(kinematic['number']), np.array(kinematic['velo'])*10)

    ax.grid()

    fig.set_figwidth(2000/fig.dpi)
    fig.set_figheight(1400/fig.dpi)
    fig.savefig(save_name+'.png')

def plot_velo_avg(left_kinematic, right_kinematic, save_name):
    fig, ax = plt.subplots()

    ax.set_title(save_name)

    ax.set_ylim(-210, 210)

    ax.set_xlabel('frame number')
    ax.set_ylabel('velocity')

    ax.set_xticks(left_kinematic['number'])
    ax.set_yticks([y*10 for y in range(-20, 21)])

    ax.plot(np.array(left_kinematic['number']), (np.array(left_kinematic['velo'])+np.array(right_kinematic['velo']))*5)

    ax.grid()

    fig.set_figwidth(2000/fig.dpi)
    fig.set_figheight(1400/fig.dpi)
    fig.savefig(save_name+'.png')

In [181]:
def plot_acc(kinematic, save_name):
    fig, ax = plt.subplots()

    ax.set_title(save_name)

    ax.set_ylim(-2000, 2000)

    ax.set_xlabel('frame number')
    ax.set_ylabel('acceleration')

    ax.set_xticks(kinematic['number'])
    ax.set_yticks([y*100 for y in range(-20, 21)])

    ax.plot(np.array(kinematic['number']), np.array(kinematic['acc'])*100)

    ax.grid()

    fig.set_figwidth(2000/fig.dpi)
    fig.set_figheight(1400/fig.dpi)
    fig.savefig(save_name+'.png')

def plot_acc_avg(left_kinematic, right_kinematic, save_name):
    fig, ax = plt.subplots()

    ax.set_title(save_name)

    ax.set_ylim(-2000, 2000)

    ax.set_xlabel('frame number')
    ax.set_ylabel('acceleration')

    ax.set_xticks(left_kinematic['number'])
    ax.set_yticks([y*100 for y in range(-20, 21)])

    ax.plot(np.array(left_kinematic['number']), (np.array(left_kinematic['acc'])+np.array(right_kinematic['acc']))*50)

    ax.grid()

    fig.set_figwidth(2000/fig.dpi)
    fig.set_figheight(1400/fig.dpi)
    fig.savefig(save_name+'.png')

In [182]:
def left_voxel_kinematic(PATH_data_list):
    data_number_list = list()
    dis = list()
    velo = list()
    acc = list()

    for PATH_data in PATH_data_list:
        data_number = PATH_data.split('/')[-1].split('.')[0].split('\\')[-1]
        data_number_list.append(int(data_number))
        
        # Load data: using CPU
        with open(PATH_data, 'rb') as f:
            data = pickle.load(f)

        feature_map = data['feature_map']
        # print(feature_map.keys())
        dis.append(feature_map['dis_to_ego'][9][14])
        velo.append(feature_map['velocity_x'][9][14])
        acc.append(feature_map['acceleration_x'][9][14])

    return {'number': data_number_list, 'dis': dis, 'velo': velo, 'acc': acc}

def right_voxel_kinematic(PATH_data_list):
    data_number_list = list()
    dis = list()
    velo = list()
    acc = list()

    for PATH_data in PATH_data_list:
        data_number = PATH_data.split('/')[-1].split('.')[0].split('\\')[-1]
        data_number_list.append(int(data_number))
        
        # Load data: using CPU
        with open(PATH_data, 'rb') as f:
            data = pickle.load(f)

        feature_map = data['feature_map']
        # print(feature_map.keys())
        dis.append(feature_map['dis_to_ego'][9][15])
        velo.append(feature_map['velocity_x'][9][15])
        acc.append(feature_map['acceleration_x'][9][15])

    return {'number': data_number_list, 'dis': dis, 'velo': velo, 'acc': acc}

In [196]:
def make_df(left, right):
    df = pd.DataFrame({
        'frame': left['number'],
        'left dis': left['dis'],
        'right dis': right['dis'],
        'avg dis': (np.array(left['dis']+np.array(right['dis']))/2),
        'left velo': np.array(left['velo'])*10,
        'right velo': np.array(right['velo'])*10,
        'avg velo': (np.array(left['velo']+np.array(right['velo']))*5),
        'left acc': np.array(left['acc'])*100,
        'right acc': np.array(right['acc'])*100,
        'avg acc': (np.array(left['acc']+np.array(right['acc']))*50),
    })

    df.to_csv('values.csv', index=False)

### Load Data Path

In [185]:
# 285~325
PATH_sample_data_TP_list = glob.glob("../../dataset/data_preprocessed/Vulner/21-12-01-11-41-59_end_extract_drive8/*")

# 300~350
PATH_sample_data_TN_list = glob.glob("../../dataset/data_preprocessed/None-crash/21-12-01-11-07-44_end_extract_drive3/*")

# 95~115
PATH_sample_data_FP_list = glob.glob("../../dataset/data_preprocessed/None-crash/21-12-29-11-04-19_end_extract_drive14/*") 

# 290~305 -> start with 292
PATH_sample_data_FN_list = glob.glob("../../dataset/data_preprocessed/Vulner/21-12-14-15-04-44_end_extract_drive2/*") 

PATH_sample_data_TN_list = PATH_sample_data_TN_list[297:347+1]
PATH_sample_data_FP_list = PATH_sample_data_FP_list[92:112+1]

model = torch.load('../../model/model.pt').cpu()

In [186]:
plot_dis(kinematic=left_voxel_kinematic(PATH_data_list=PATH_sample_data_TP_list), save_name='Left Voxel Distance')
plot_dis(kinematic=right_voxel_kinematic(PATH_data_list=PATH_sample_data_TP_list), save_name='Right Voxel Distance')
plot_dis_avg(left_kinematic=left_voxel_kinematic(PATH_data_list=PATH_sample_data_TP_list),
             right_kinematic=right_voxel_kinematic(PATH_data_list=PATH_sample_data_TP_list),
             save_name='Average Distance')

plot_velo(kinematic=left_voxel_kinematic(PATH_data_list=PATH_sample_data_TP_list), save_name='Left Voxel Velocity')
plot_velo(kinematic=right_voxel_kinematic(PATH_data_list=PATH_sample_data_TP_list), save_name='Right Voxel Velocity')
plot_velo_avg(left_kinematic=left_voxel_kinematic(PATH_data_list=PATH_sample_data_TP_list),
             right_kinematic=right_voxel_kinematic(PATH_data_list=PATH_sample_data_TP_list),
             save_name='Average Velocity')

plot_acc(kinematic=left_voxel_kinematic(PATH_data_list=PATH_sample_data_TP_list), save_name='Left Voxel Acceleration')
plot_acc(kinematic=right_voxel_kinematic(PATH_data_list=PATH_sample_data_TP_list), save_name='Right Voxel Acceleration')
plot_acc_avg(left_kinematic=left_voxel_kinematic(PATH_data_list=PATH_sample_data_TP_list),
             right_kinematic=right_voxel_kinematic(PATH_data_list=PATH_sample_data_TP_list),
             save_name='Average Acceleration')

In [197]:
make_df(left=left_voxel_kinematic(PATH_data_list=PATH_sample_data_TP_list),
        right=right_voxel_kinematic(PATH_data_list=PATH_sample_data_TP_list))